<a href="https://colab.research.google.com/github/DorotaJanosz/machine-learning-bootcamp/blob/master/unsupervised/02_dimensionality_reduction/03_pca_wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import bibliotek

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

np.set_printoptions(precision=6, suppress=True, edgeitems=5, linewidth=200)

###Załadowanie danych

In [3]:
df_raw = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df = df_raw.copy()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [10]:
data = df.iloc[:, 1:]
target = df.iloc[:, 0]
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [12]:
target.head()

0    1
1    1
2    1
3    1
4    1
Name: 0, dtype: int64

In [13]:
target.value_counts()

2    71
1    59
3    48
Name: 0, dtype: int64

###Podział na biór treningowy i testowy

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target)
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

X_train shape: (133, 13)
X_test shape: (45, 13)


###Standaryzacja

In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)
X_train_std[:5]


array([[ 1.6637  , -0.617986,  1.216519,  1.572763, -0.149108,  0.826171, -0.650403,  1.284581,  1.904715,  3.367056, -1.619543, -0.846402, -0.230492],
       [ 0.80253 , -0.498787,  1.216519, -0.71077 ,  0.811581,  0.903421,  0.922568, -0.517131, -0.204077,  0.944969,  1.44796 ,  0.420457,  1.969581],
       [-0.725743, -1.177303, -0.959147, -0.330181, -0.835315,  1.938575,  1.748379, -0.987143,  0.629236, -0.244829,  0.411058,  0.268985, -0.230492],
       [ 1.045113, -0.718846,  0.92643 ,  0.10896 ,  1.017443,  1.057922,  1.404291,  0.266222,  0.238089,  0.647519,  0.799896, -0.006419,  1.36033 ],
       [-0.774259, -1.250656, -1.503063, -1.413395,  2.458477, -0.564333, -0.119525, -0.125454,  2.006754, -0.712249,  0.497466, -0.364445,  0.074133]])

###PCA

In [34]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)
X_train_pca.shape

(133, 3)

###Wyjaśniona wariancja

In [31]:
results = pd.DataFrame(data={'explained_variance_ratio': pca.explained_variance_ratio_})
results['cumulative'] = results['explained_variance_ratio'].cumsum()
results['component'] = results.index + 1
results

,explained_variance_ratio,cumulative,component
0,0.358956,0.358956,1
1,0.187999,0.546954,2
2,0.115944,0.662898,3


In [32]:
fig = go.Figure(data=[go.Bar(x=results['component'], y=results['explained_variance_ratio'], name='explained variance ratio'),
                      go.Scatter(x=results['component'], y=results['cumulative'], name='cumulative explained variance')],
                layout=go.Layout(title=f'PCA - {pca.n_components_} components', width=950, template='plotly_dark'))
fig.show()

###Wykres

In [38]:
X_train_pca_df = pd.DataFrame(np.c_[X_train_pca, y_train], columns=['pca_1', 'pca_2', 'pca_3', 'target'])
X_train_pca_df.head()

,pca_1,pca_2,pca_3,target
0,-0.990347,-3.632632,1.118362,3.0
1,2.382431,-1.700685,-0.177707,1.0
2,2.302863,1.337646,-0.025856,2.0
3,2.000502,-1.693119,0.441186,1.0
4,1.434157,1.012820,-1.327788,2.0


In [40]:
px.scatter_3d(data_frame=X_train_pca_df, x='pca_1', y='pca_2', z='pca_3', color='target', template='plotly_dark', width=950)